# ETL Pipeline for College Rate of Return Analysis



In [1]:
import pandas as pd
import re

## Extract
Read the data into python using the pandas library.

In [2]:
salariesBycollegeCSV = "Resources/DS_WhereItPaysToAttendCollege/salaries-by-college-type.csv"
salariesBycollege_df = pd.read_csv(salariesBycollegeCSV)

college_AccTuitionCSV = "Resources/DS_USWorldReportCollegeData/College.csv"
college_AccTuition_df = pd.read_csv(college_AccTuitionCSV)

usa_rankingCSV = "Resources/NationalUniversitiesRankings.csv"
usa_ranking_df = pd.read_csv(usa_rankingCSV, encoding = "latin1")

The data file below is brought in to clean up NaN values later in the Transform section.

In [3]:
#bring in CSV for grabbing missing Acceptance rate for NaN values
#mostRecentDataCSV = "Resources/Most-Recent-Cohorts-All-Data-Elements.csv"
mostRecentDataCSV = "Resources/DS_collegescorecard_ed_gov/Most-Recent-Cohorts-All-Data-Elements.csv"
mostRecentData_df = pd.read_csv(mostRecentDataCSV,error_bad_lines=False, index_col=False, dtype='unicode')
#Grab the columns for admission rate and median earnings
mostADMrate_MDsalary_df = mostRecentData_df.loc[:,['INSTNM','ADM_RATE_ALL','MD_EARN_WNE_P10']]
mostADMrate_MDsalary_df.head()

,INSTNM,ADM_RATE_ALL,MD_EARN_WNE_P10
0,Alabama A & M University,0.87377537980974,31000
1,University of Alabama at Birmingham,0.581351094196,41200
2,Amridge University,NaN,39600
3,University of Alabama in Huntsville,0.76281628162816,46700
4,Alabama State University,0.458963879377,27700


## Transform
* Remove columns that are not needed.
* Add a column to merge on that removes spaces and special characters and makes everything lowercase.
* Merge the tables together and clean up NaN values.

In [4]:
salariesBycollege_df.head()
#School Name, Starting Median Salary, Mid-Career Median Salary



salariesBycollege_df = salariesBycollege_df.loc[:, ['School Name', 'Starting Median Salary', 'Mid-Career Median Salary']]
salariesBycollege_df.head()

#salariesBycollege_df.describe #[269 rows x 3 columns]>


,School Name,Starting Median Salary,Mid-Career Median Salary
0,Massachusetts Institute of Technology (MIT),"$72,200.00","$126,000.00"
1,California Institute of Technology (CIT),"$75,500.00","$123,000.00"
2,Harvey Mudd College,"$71,800.00","$122,000.00"
3,"Polytechnic University of New York, Brooklyn","$62,400.00","$114,000.00"
4,Cooper Union,"$62,200.00","$114,000.00"


In [5]:
#Creating a column to merge on

for i, row in salariesBycollege_df.iterrows():
    
    #replaces () with nothing.. reference here: 
    #https://stackoverflow.com/questions/20894525/how-to-remove-parentheses-and-all-data-within-using-pandas-python
    salariesBycollege_df.loc[i, 'School Name'] = re.sub(r" ?\([^)]+\)", "", salariesBycollege_df.loc[i, 'School Name'])
    #Create a new column to remove spaces and special characters
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'School Name']
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].replace('-', '')
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].replace(' ', '')
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].replace('.', '')
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].replace('\'', '')
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].replace('&', '')
    salariesBycollege_df.loc[i, 'merge_name'] = salariesBycollege_df.loc[i, 'merge_name'].lower()
    
salariesBycollege_df.head()

,School Name,Starting Median Salary,Mid-Career Median Salary,merge_name
0,Massachusetts Institute of Technology,"$72,200.00","$126,000.00",massachusettsinstituteoftechnology
1,California Institute of Technology,"$75,500.00","$123,000.00",californiainstituteoftechnology
2,Harvey Mudd College,"$71,800.00","$122,000.00",harveymuddcollege
3,"Polytechnic University of New York, Brooklyn","$62,400.00","$114,000.00","polytechnicuniversityofnewyork,brooklyn"
4,Cooper Union,"$62,200.00","$114,000.00",cooperunion


In [6]:

#name, Private, Apps, Accept "AcpRate"
college_AccTuition_df.rename( columns={'Unnamed: 0':'Name'}, inplace=True )

college_accept_df = college_AccTuition_df.loc[:, ['Name', 'Private', 'Apps', 'Accept']]

college_accept_df['Acp_Rate(%)'] = round(college_accept_df['Accept'] / college_accept_df['Apps'] * 100,2)

college_accept_df.head()



,Name,Private,Apps,Accept,Acp_Rate(%)
0,Abilene Christian University,Yes,1660,1232,74.22
1,Adelphi University,Yes,2186,1924,88.01
2,Adrian College,Yes,1428,1097,76.82
3,Agnes Scott College,Yes,417,349,83.69
4,Alaska Pacific University,Yes,193,146,75.65


In [7]:
#Creating a column to merge on
for i, row in college_accept_df.iterrows():
    
    #Create a new column to remove spaces and special characters
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'Name']
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].replace('-', '')
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].replace(' ', '')
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].replace('.', '')
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].replace('\'', '')
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].replace('&', '')
    college_accept_df.loc[i, 'merge_name'] = college_accept_df.loc[i, 'merge_name'].lower()

college_accept_df.head()

,Name,Private,Apps,Accept,Acp_Rate(%),merge_name
0,Abilene Christian University,Yes,1660,1232,74.22,abilenechristianuniversity
1,Adelphi University,Yes,2186,1924,88.01,adelphiuniversity
2,Adrian College,Yes,1428,1097,76.82,adriancollege
3,Agnes Scott College,Yes,417,349,83.69,agnesscottcollege
4,Alaska Pacific University,Yes,193,146,75.65,alaskapacificuniversity


In [8]:
usa_ranking_df.head()
#Name, Location, Rank, Tuition and fees
usa_ranking_df = usa_ranking_df.loc[:,['Name', 'Location', 'Rank', 'Tuition and fees']]
usa_ranking_df.head()



,Name,Location,Rank,Tuition and fees
0,Princeton University,"Princeton, NJ",1,"$45,320"
1,Harvard University,"Cambridge, MA",2,"$47,074"
2,University of Chicago,"Chicago, IL",3,"$52,491"
3,Yale University,"New Haven, CT",3,"$49,480"
4,Columbia University,"New York, NY",5,"$55,056"


In [9]:
#Create a column to merge on
for i, row in usa_ranking_df.iterrows():
    
    #Create a new column to remove spaces and special characters
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'Name']
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].replace('-', '')
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].replace(' ', '')
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].replace('.', '')
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].replace('\'', '')
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].replace('&', '')
    usa_ranking_df.loc[i, 'merge_name'] = usa_ranking_df.loc[i, 'merge_name'].lower()

usa_ranking_df.head()

,Name,Location,Rank,Tuition and fees,merge_name
0,Princeton University,"Princeton, NJ",1,"$45,320",princetonuniversity
1,Harvard University,"Cambridge, MA",2,"$47,074",harvarduniversity
2,University of Chicago,"Chicago, IL",3,"$52,491",universityofchicago
3,Yale University,"New Haven, CT",3,"$49,480",yaleuniversity
4,Columbia University,"New York, NY",5,"$55,056",columbiauniversity


### Start Merging

In [10]:
merged_df = usa_ranking_df.merge(college_accept_df, on='merge_name', how='left')
all_merged_df = merged_df.merge(salariesBycollege_df, on='merge_name', how='left')
all_merged_df.rename(columns={'Name_x': 'Name'}, inplace=True)
all_merged_df.drop(columns=['Name_y', 'School Name'], inplace=True)
all_merged_df

,Name,Location,Rank,Tuition and fees,merge_name,Private,Apps,Accept,Acp_Rate(%),Starting Median Salary,Mid-Career Median Salary
0,Princeton University,"Princeton, NJ",1,"$45,320",princetonuniversity,Yes,13218.0,2042.0,15.45,"$66,500.00","$131,000.00"
1,Harvard University,"Cambridge, MA",2,"$47,074",harvarduniversity,Yes,13865.0,2165.0,15.61,"$63,400.00","$124,000.00"
2,University of Chicago,"Chicago, IL",3,"$52,491",universityofchicago,Yes,6348.0,2999.0,47.24,NaN,NaN
3,Yale University,"New Haven, CT",3,"$49,480",yaleuniversity,Yes,10705.0,2453.0,22.91,"$59,100.00","$126,000.00"
4,Columbia University,"New York, NY",5,"$55,056",columbiauniversity,Yes,6756.0,1930.0,28.57,"$59,400.00","$107,000.00"
5,Stanford University,"Stanford, CA",5,"$47,940",stanforduniversity,NaN,NaN,NaN,NaN,NaN,NaN
6,Massachusetts Institute of Technology,"Cambridge, MA",7,"$48,452",massachusettsinstituteoftechnology,Yes,6411.0,2140.0,33.38,"$72,200.00","$126,000.00"
7,Duke University,"Durham, NC",8,"$51,265",dukeuniversity,Yes,13789.0,3893.0,28.23,NaN,NaN
8,University of Pennsylvania,"Philadelphia, PA",8,"$51,464",universityofpennsylvania,Yes,12394.0,5232.0,42.21,"$60,900.00","$120,000.00"
9,Johns Hopkins University,"Baltimore, MD",10,"$50,410",johnshopkinsuniversity,Yes,8474.0,3446.0,40.67,NaN,NaN


In [11]:
# Check for NaN values
all_merged_df.isnull().sum()

Name                          0
Location                      0
Rank                          0
Tuition and fees              0
merge_name                    0
Private                     123
Apps                        123
Accept                      123
Acp_Rate(%)                 123
Starting Median Salary      144
Mid-Career Median Salary    144
dtype: int64

### Data improvement from another resource in effort to reduce the amount of "NaN" values

In [12]:
#Creating a column to merge on
for i, row in mostADMrate_MDsalary_df.iterrows():
    
    #Create a new column to remove spaces and special characters
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'INSTNM']
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].replace('-', '')
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].replace(' ', '')
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].replace('.', '')
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].replace('\'', '')
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].replace('&', '')
    mostADMrate_MDsalary_df.loc[i, 'merge_name'] = mostADMrate_MDsalary_df.loc[i, 'merge_name'].lower()
    
mostADMrate_MDsalary_df.head()

,INSTNM,ADM_RATE_ALL,MD_EARN_WNE_P10,merge_name
0,Alabama A & M University,0.87377537980974,31000,alabamaamuniversity
1,University of Alabama at Birmingham,0.581351094196,41200,universityofalabamaatbirmingham
2,Amridge University,NaN,39600,amridgeuniversity
3,University of Alabama in Huntsville,0.76281628162816,46700,universityofalabamainhuntsville
4,Alabama State University,0.458963879377,27700,alabamastateuniversity


In [13]:
#get list of schools with NaN values in the salary column

missingSalaryDatalist = []

missingSalaryData = pd.isnull(all_merged_df['Starting Median Salary'])

for i, row in enumerate(missingSalaryData):
    if row == True:
        
        for x, stuff in all_merged_df.iterrows():
            if x == i:
                #print(stuff['Name'])
                missingSalaryDatalist.append(stuff['merge_name'])
                
print(len(missingSalaryDatalist))

144


In [14]:
#get list of schools with NaN values in the admission rate column

missingAdmDatalist = []

missingAdmData = pd.isnull(all_merged_df['Acp_Rate(%)'])

for i, row in enumerate(missingAdmData):
    if row == True:
        
        for x, stuff in all_merged_df.iterrows():
            if x == i:
                #print(stuff['Name'])
                missingAdmDatalist.append(stuff['merge_name'])
                
print(len(missingAdmDatalist))

123


In [15]:
#Grab missing salary data from mostADMrate_MDsalary_df
missingMDsalary = {}

for i, row in mostADMrate_MDsalary_df.iterrows():
    
    for line in missingSalaryDatalist:
        
        if row[3] == line:
            #Format to currency while pulling data
            #Some rows had the string 'PrivacySuppressed' instead of a number - replace with NaN
            if row[2] == 'PrivacySuppressed':
                missingMDsalary.update({line: 'NaN'})
            elif row[2][0] != '$':
                row[2] = float(row[2])
                row[2] = '${:,.2f}'.format(row[2])
                missingMDsalary.update({line:row[2]})
            else:
                missingMDsalary.update({line:row[2]})

#remove added NaN values
update_missingMDsalary = {}
for key in missingMDsalary:
    if missingMDsalary[key] != 'NaN':
        update_missingMDsalary[key] = missingMDsalary[key]
        
print(len(update_missingMDsalary))

111


In [16]:
#Grab missing admission rate from mostADMrate_MDsalary_df

missingADMrate = {}

for i, row in mostADMrate_MDsalary_df.iterrows():
    
    for line in missingAdmDatalist:
        
        if row[3] == line:
            row[1] = float(row[1])
            row[1] = round(row[1] * 100,2)
            missingADMrate.update({line:row[1]})
            
if 'Benedictine University' in missingADMrate: 
    del missingADMrate['Benedictine University']
            
print(len(missingADMrate))

84


In [17]:
#if Acp_Rate(%) has a NaN value and a new value in missingADMrate, map and replace the NaN value
all_merged_df.loc[all_merged_df['Acp_Rate(%)'].isnull(), 'Acp_Rate(%)'] = all_merged_df['merge_name'].map(missingADMrate)
all_merged_df.loc[all_merged_df['Starting Median Salary'].isnull(), 'Starting Median Salary'] = all_merged_df['merge_name'].map(update_missingMDsalary)



### Final result of cleaning up NaN values
* Roughly 75% of the data was recovered for the Acp_Rate and Staring Salary columns

In [18]:
all_merged_df.isnull().sum()

Name                          0
Location                      0
Rank                          0
Tuition and fees              0
merge_name                    0
Private                     123
Apps                        123
Accept                      123
Acp_Rate(%)                  38
Starting Median Salary       33
Mid-Career Median Salary    144
dtype: int64

In [19]:
#final formating to remove merge_name column
all_merged_df.drop(columns=['merge_name'], inplace=True)
all_merged_df

,Name,Location,Rank,Tuition and fees,Private,Apps,Accept,Acp_Rate(%),Starting Median Salary,Mid-Career Median Salary
0,Princeton University,"Princeton, NJ",1,"$45,320",Yes,13218.0,2042.0,15.45,"$66,500.00","$131,000.00"
1,Harvard University,"Cambridge, MA",2,"$47,074",Yes,13865.0,2165.0,15.61,"$63,400.00","$124,000.00"
2,University of Chicago,"Chicago, IL",3,"$52,491",Yes,6348.0,2999.0,47.24,"$68,100.00",NaN
3,Yale University,"New Haven, CT",3,"$49,480",Yes,10705.0,2453.0,22.91,"$59,100.00","$126,000.00"
4,Columbia University,"New York, NY",5,"$55,056",Yes,6756.0,1930.0,28.57,"$59,400.00","$107,000.00"
5,Stanford University,"Stanford, CA",5,"$47,940",NaN,NaN,NaN,4.81,"$94,000.00",NaN
6,Massachusetts Institute of Technology,"Cambridge, MA",7,"$48,452",Yes,6411.0,2140.0,33.38,"$72,200.00","$126,000.00"
7,Duke University,"Durham, NC",8,"$51,265",Yes,13789.0,3893.0,28.23,"$84,400.00",NaN
8,University of Pennsylvania,"Philadelphia, PA",8,"$51,464",Yes,12394.0,5232.0,42.21,"$60,900.00","$120,000.00"
9,Johns Hopkins University,"Baltimore, MD",10,"$50,410",Yes,8474.0,3446.0,40.67,"$73,200.00",NaN


## Load
* Write the final pandas dataframe into a CSV file.
* The final table can be now be used for visual analysis for which of the top ranked universities/college give you the best value

In [20]:
all_merged_df.to_csv('Results/Final_College_Data.csv', encoding='utf-8')